In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import BooleanType
#from rateFunction import *
spark = SparkSession.builder.appName("Ngram_buzzwords").master("spark://192.168.1.76:7077").getOrCreate()
sc = spark.sparkContext

In [2]:
def sortBuzzWords(buzzwords, index_value):
    return sorted(buzzwords, key=lambda x: x[index_value], reverse=True)

In [14]:
df = spark.read.csv('../../data/eng-gb-all-only-letters.csv', sep="\t", header=False)

In [15]:
df = df.withColumnRenamed('_c0', 'ngram')
df = df.withColumnRenamed('_c1', 'year')
df = df.withColumnRenamed('_c2', 'match_count')
df = df.withColumnRenamed('_c3', 'page_count')
df = df.withColumnRenamed('_c4', 'volume_count')

In [16]:
totalCountsDf = spark.read.csv('../../data/eng-gb-all/googlebooks-eng-gb-all-totalcounts-20090715.txt', sep="\t", header=False)

In [17]:
totalCountsDf = totalCountsDf.withColumnRenamed('_c0', 'year')
totalCountsDf = totalCountsDf.withColumnRenamed('_c1', 'match_count')
totalCountsDf = totalCountsDf.withColumnRenamed('_c2', 'page_count')
totalCountsDf = totalCountsDf.withColumnRenamed('_c3', 'volume_count')

In [18]:
totalCounts = sc.broadcast(totalCountsDf.collect())

In [23]:
totalCountsRDD = totalCountsDf.rdd.map(list)

In [27]:
totalCounts = sc.broadcast(dict(totalCountsRDD.map(lambda x: (x[0], (x[1],x[2],x[3]))).collect()))

In [30]:
totalCounts.value["1944"]

('72792722', '169964', '781')

In [31]:
totalCounts.value

{'1541': ('5056', '27', '1'),
 '1547': ('47904', '266', '1'),
 '1551': ('66807', '168', '1'),
 '1562': ('132418', '451', '1'),
 '1576': ('26278', '81', '1'),
 '1582': ('27957', '124', '1'),
 '1584': ('31257', '92', '1'),
 '1588': ('78847', '243', '1'),
 '1590': ('49862', '141', '1'),
 '1591': ('16332', '65', '1'),
 '1592': ('34054', '144', '3'),
 '1593': ('69633', '360', '3'),
 '1594': ('78034', '283', '4'),
 '1595': ('8312', '62', '1'),
 '1596': ('93900', '359', '3'),
 '1597': ('163424', '666', '3'),
 '1598': ('109693', '373', '3'),
 '1599': ('86102', '424', '3'),
 '1600': ('81567', '312', '5'),
 '1602': ('16426', '80', '2'),
 '1603': ('247134', '1150', '2'),
 '1605': ('33862', '57', '1'),
 '1606': ('63033', '297', '3'),
 '1607': ('25021', '100', '1'),
 '1608': ('119151', '464', '4'),
 '1609': ('191075', '564', '9'),
 '1611': ('13618', '47', '1'),
 '1612': ('90439', '289', '2'),
 '1613': ('31745', '126', '1'),
 '1615': ('9065', '54', '1'),
 '1617': ('98765', '430', '1'),
 '1619': ('27

In [5]:
#df.count()

In [6]:
rdd = df.rdd.map(list)

In [8]:
def splitAndMakeNgramKey(row):
    elements = row[0].split(",")
    return(elements[0], (elements[1:len(elements)]))

rdd = rdd.map(splitAndMakeNgramKey)

In [9]:
#rdd.take(10)

In [10]:
#rdd2 = sc.parallelize(rdd.take(1000000))

In [11]:
def groupData(a,b):
    if isinstance(a[0], (list,)):
        return a+[b]
    else:
        return [a]+[b]

rdd2 = rdd.reduceByKey(groupData)

In [13]:
def yearRate(word):
    if len(word) == 0:
        return [('0',0,0.0,0,0,0.0)]
    
    if len(word) == 1:
        return [(word[0][0], 0, 0.0,0,0,0.0)]
    
    wordArray=[]
    for i in range(len(word)-1):
        try:
            yearPair = str(word[i][0])+"-"+str(word[i+1][0])
            diff = int(word[i+1][1])-int(word[i][1])
            diffProcent = (float(word[i+1][1])/float(word[i][1]))*100
            buzzWordValue = diff*diffProcent
            wordArray.append((yearPair,diff,diffProcent,int(word[i][1]),int(word[i+1][1]),buzzWordValue))
        except:
            return [('0',0,0.0,0,0,0.0)]
    return wordArray

In [14]:
rdd3 = rdd2.map(lambda row: (row[0], yearRate(row[1])))

In [15]:
#rdd_count = rdd3.map(lambda row: (row[0], sortBuzzWords(row[1], 1)))

In [16]:
#rdd_percent = rdd3.map(lambda row: (row[0], sortBuzzWords(row[1], 2)))

In [17]:
rdd_buzzword = rdd3.map(lambda row: (row[0], sortBuzzWords(row[1], 5)))

In [18]:
#rdd_count2 = rdd_count.sortBy(lambda row: row[1][0][1], ascending=False)

In [19]:
#rdd_percent2 = rdd_percent.sortBy(lambda row: row[1][0][2], ascending=False)

In [20]:
rdd_buzzword2 = rdd_buzzword.sortBy(lambda row: row[1][0][5], ascending=False)

In [ ]:
#rdd_count2.take(10)

In [ ]:
#rdd_percent2.take(10)

In [ ]:
rdd_buzzword2.take(1000)

In [ ]:
#rdd3.coalesce(1).saveAsTextFile("Words-with-year-list.txt")